# Building a Spotify Playlist Track Recommender using a Graph Neural Network approch

## Introduction:
In this tutorial we will be seeing how to use a Graph Neural Network approch to build a Spotify Playlist Track Recommender. 
The ask is automatic playlist continuation: given an initial set of tracks in a playlist, predict the subsequent tracks in that playlist.
Thus the goal is to be able to recommend songs that should be added to a playlist.
## Dataset:
We will be working on the dataset from the Spotify Million Playlist Dataset Challenge. The Spotify Million Playlist Dataset Challenge consists of a dataset and evaluation to enable research in music recommendations. It is a continuation of the RecSys Challenge 2018.  
The dataset contains 1,000,000 playlists and over 2 million unique tracks by nearly 300,000 artists including playlist titles and track titles, created by users on the Spotify platform between January 2010 and October 2017. It represents the largest public dataset of music playlists in the world.
## Metrics:
We will be using the Recall@k metric that can be calculated for every playlist:  
$$Recall@k = \frac{|songs\,in\,the\,top\,k\,recommendations \cap ground\,truth |}{|ground\,truth |}$$   

## 1. Importing and installing the necessary modules

In [ ]:
! pip install --upgrade scipy networkx
! pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
! pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
! pip install -q torch-geometric
! pip install aicrowd-cli

In [ ]:
# File management
import os
import json
import sys
# Data manipulation
import numpy as np
# Torch
import torch
# Graph manipulation
import networkx as nx
# Torch geometric
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn.models import LightGCN
# Reporting
from tqdm.notebook import tqdm
from time import time
import datetime as dtime
# Warning
import warnings
warnings.filterwarnings('ignore', category = UserWarning) # We're ignoring a user warning during training to make reporting a little cleaner.

## 2. Downloading the dataset
In September 2020, Spotify re-released the dataset as an open-ended challenge on AIcrowd.com. The dataset can now be downloaded by registered participants from the Resources page. 
To download the dataset, we will use AICrowd CLI library.

**Note: An AICrowd account is required to login and obtainan API key.**

In [ ]:
! aicrowd login

Please login here: https://api.aicrowd.com/auth/oyOqvxpTv3mPsyTI4FmB76ZlhV5zJcix12LW-15Yiw8
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/oyOqvxpTv3mPsyTI4FmB76ZlhV5zJcix12LW-15Yiw8'
API Key valid
Gitlab access token valid
Saved details successfully!


The Spotify million playlist challenge contains multiple files. The first `zip` file is what we will use to make our dataset. The other files are for submission, which isn't our objective for now.

In [ ]:
! aicrowd dataset list -c 'https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge'

                          Datasets for challenge #277                           
┌───┬────────────────────────────────────────────────┬─────────────┬───────────┐
│ # │ Title                                          │ Description │      Size │
├───┼────────────────────────────────────────────────┼─────────────┼───────────┤
│ 0 │ Resourcesspotify_million_playlist_dataset.zip  │ -           │   5.79 GB │
│ 1 │ Resourcesspotify_million_playlist_dataset_cha… │ -           │ 113.33 MB │
└───┴────────────────────────────────────────────────┴─────────────┴───────────┘


In [ ]:
! mkdir /content/logs
! aicrowd dataset download -c 'https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge' '*spotify_million_playlist_dataset.zip' -o /content -j 1 > /content/logs/download.txt

spotify_million_playlist_dataset.zip: 100% 5.79G/5.79G [05:37<00:00, 17.2MB/s]


In [ ]:
! unzip -qq /content/spotify_million_playlist_dataset.zip
! rm /content/spotify_million_playlist_dataset.zip

## 3. Data overview

The file `stats.txt` already contains some interesting statistics about the dataset. We will, therefore, read the file and make some observations.

In [ ]:
!cat /content/stats.txt | head -10 | tail -9

number of playlists 1000000
number of tracks 66346428
number of unique tracks 2262292
number of unique albums 734684
number of unique artists 295860
number of unique titles 92944
number of playlists with descriptions 18760
number of unique normalized titles 17381
avg playlist length 66.346428


In [ ]:
!cat /content/stats.txt | tail -154 | head -21


top playlist titles
  10000 country
  10000 chill
   8493 rap
   8481 workout
   8146 oldies
   8015 christmas
   6848 rock
   6157 party
   5883 throwback
   5063 jams
   5052 worship
   4907 summer
   4677 feels
   4612 new
   4186 disney
   4124 lit
   4030 throwbacks
   3886 music
   3513 sleep


In [ ]:
!cat /content/stats.txt | tail -131 | head -21

top tracks
  46574 HUMBLE. by Kendrick Lamar
  43447 One Dance by Drake
  41309 Broccoli (feat. Lil Yachty) by DRAM
  41079 Closer by The Chainsmokers
  39987 Congratulations by Post Malone
  35202 Caroline by Aminé
  35138 iSpy (feat. Lil Yachty) by KYLE
  34999 Bad and Boujee (feat. Lil Uzi Vert) by Migos
  34990 Location by Khalid
  34922 XO TOUR Llif3 by Lil Uzi Vert
  33699 Bounce Back by Big Sean
  32391 Ignition - Remix by R. Kelly
  32336 No Role Modelz by J. Cole
  32059 Mask Off by Future
  31492 No Problem (feat. Lil Wayne & 2 Chainz) by Chance The Rapper
  31374 I'm the One by DJ Khaled
  31119 Jumpman by Drake
  31106 goosebumps by Travis Scott
  30678 Fake Love by Drake
  30485 Despacito - Remix by Luis Fonsi


In [ ]:
!cat /content/stats.txt | tail -109 | head -21

top artists
 847160 Drake
 413297 Kanye West
 353624 Kendrick Lamar
 339570 Rihanna
 316603 The Weeknd
 294667 Eminem
 272116 Ed Sheeran
 250734 Future
 243119 Justin Bieber
 241560 J. Cole
 230857 Beyoncé
 223509 The Chainsmokers
 212772 Chris Brown
 203047 Calvin Harris
 198905 Twenty One Pilots
 197855 Lil Uzi Vert
 195907 Post Malone
 192478 Big Sean
 187029 Maroon 5
 185520 JAY Z


In [ ]:
!cat /content/stats.txt | tail -43 | head -21

playlist length histogram
  15057 20
  14177 15
  13876 21
  13856 16
  13685 17
  13629 18
  13602 22
  13531 19
  13250 24
  13149 23
  13077 30
  13043 14
  13031 25
  12834 26
  12513 28
  12502 27
  12332 29
  12318 13
  12016 12
  11882 31


In [ ]:
!cat /content/stats.txt | tail -21

num followers histogram
 754219 1
 149600 2
  46939 3
  19591 4
   9813 5
   5360 6
   3305 7
   2143 8
   1512 9
   1006 10
    825 11
    632 12
    479 13
    359 14
    328 15
    290 16
    235 17
    207 18
    162 19
    138 20


## 4. Data preprocessing
Since the spotify dataset is too large, we will need to cut down the dataset so we can use it in train with our computation power. The idea is to use only playlists that have a certain number of followers. We suppose that playlists with a large number of followers can relate to a bigger amount of users. For this we use Networkx.  
In order to feed the data to a GNN, it needs to respect a specific format. Hence, we present a couple of helper functions that process data and transform it to graph in the COO format.

**Note: Using an adjacency matrix to represent graphs is usually impractical. Adjacency matrices are usually large and require a significant amount of RAM to carry. Besides, adjacency matrices are usually sparse (contain a lot of 0s). Hence, occupying a lot of storage to store null values is inconvenient.**

In [ ]:
def count_playlists(path: str, min_followers: int):
    """
    Checks the number of playlists having more followers than min_followers.
        Parameters:
            path(str): path to the playlists folder.
            min_followers(int): minimum number of followers needed for a playlist to be counted.
        Returns:
            count(int): number of playlists having more than min_followers.
    """
    count = 0
    files = os.listdir(path)
    for file in files:
        with open(os.path.join(path, file), 'r') as f:
            playlists = json.load(f)['playlists']
            for playlist in playlists:
                if playlist['num_followers'] > min_followers:
                    count += 1
    return count

In [ ]:
def preprocess(source_folder, output_folder, min_followers):
    playlist_id = 0
    song_id = count_playlists(source_folder, min_followers)
    playlist_dict = {}
    song_dict = {}
    G = nx.Graph()

    with tqdm(os.listdir(source_folder)) as files:
        for file in files:
            with open(os.path.join(source_folder, file), 'r') as f:
                playlists = json.load(f)['playlists']
                for playlist in playlists:
                    if playlist['num_followers'] > min_followers:
                        G.add_node(playlist_id)
                        tmp = {k: v for k, v in playlist.items() if k not in ['tracks', 'num_edits', 'modified_at']}
                        tmp['id'] = playlist_id
                        playlist_dict[playlist['pid']] = tmp
                        for song in playlist['tracks']:
                            track_uri = song['track_uri']
                            if track_uri not in song_dict:
                                G.add_node(song_id)
                                tmp = {k: v for k, v in song.items()}
                                tmp['id'] = song_id
                                song_dict[song['track_uri']] = tmp
                                song_id += 1
                            G.add_edge(playlist_id, song_dict[track_uri]['id'])
                        playlist_id += 1

    edge_list = []
    for edge in G.edges:
        edge_list.append(list(edge))
        edge_list.append(list(edge)[::-1])
    edge_index = torch.LongTensor(edge_list)
    data = Data(edge_index=edge_index.t().contiguous(), num_nodes=len(G.nodes))
    torch.save(data, os.path.join(output_folder, 'graph_object.pt'))
    with open(os.path.join(output_folder, 'playlist_dict.json'), 'w') as f:
        json.dump(playlist_dict, f)
    with open(os.path.join(output_folder, 'song_dict.json'), 'w') as f:
        json.dump(song_dict, f)

In [ ]:
! mkdir /content/output
preprocess('/content/data', '/content/output', 5)

  0%|          | 0/1000 [00:00<?, ?it/s]

Now that we have processed data in the correct format to feed to a GNN model, it's about time we create our data loaders for batch handling.

## DataLoader

Let's start off by loading the processed data.

In [ ]:
data = torch.load('/content/output/graph_object.pt')
with open('/content/output/playlist_dict.json', 'r') as f:
  playlist_dict = json.load(f)
with open('/content/output/song_dict.json', 'r') as f:
  song_dict = json.load(f)

Like traditional ML, we split our data into train, validation and test splits. In graph language, this means splitting the edges into different sets since this is an edge-prediction task. We will use Torch Geometric's pre-defined function `RandomLinkSplit`.

In [ ]:
split = RandomLinkSplit(num_val = .15, num_test = .15, is_undirected = True, add_negative_train_samples = False, neg_sampling_ratio = 0)
train_split, val_split, test_split = split(data)

To train a GNN model, we use to different types of edges:
- Evaluation edges → used for loss calculation
- Message passing edges → used for message propagation and act as ground truth during evaluation

Evaluation edges will be wrapped inside a Torch Geometric Dataset class, while message passing edges will be wrapped inside a simple Torch Geometric Data class.

The Torch Geometric Dataset class is defined as follows,

In [ ]:
class PlainData(Data):
    def __inc__(self, key, value, *args, **kwargs):
        return 0

class SpotifyDataset(Dataset):
  def __init__(self, root, edge_index, transform = None, pre_transform = None, pre_filter = None):
    self.edge_index = edge_index
    self.unique_playlists = torch.unique(edge_index[0, :]).tolist()
    self.num_nodes = len(self.unique_playlists)
    super().__init__(root, transform, pre_transform, pre_filter)

  def len(self):
    return self.num_nodes

  def get(self, idx):
    edge_index = self.edge_index[:, self.edge_index[0, :] == idx] # Return all outgoing edges
    return PlainData(edge_index = edge_index) # Maybe PlainData IDK

In [ ]:
train_ev = SpotifyDataset('root', edge_index = train_split.edge_label_index) # 'root' is a random string
train_mp = Data(edge_index = train_split.edge_index)
val_ev = SpotifyDataset('root', edge_index = val_split.edge_label_index)
val_mp = Data(edge_index = val_split.edge_index)

Finally we prepare our loader. Message passing edges do not require batching, only evaluation edges will be loaded in batches.

In [ ]:
batch_size = 512 # Feel free to experiment and change this hyper-parameter

train_loader = DataLoader(train_ev, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_ev, batch_size = batch_size, shuffle = False)

## Negative sampling

Our data, so far, contains only positive edges (edges that represent actual links between edges). In order to train the model, we need to expose it to correct links and false links (negative samples). Therefore, we need to generate these false links.
Below is a helper function that samples negative edges.

In [ ]:
def sample_negative_edges(batch, num_playlists, num_nodes):
    negs = []
    for i in batch.edge_index[0,:]:
        assert i < num_playlists
        rand = torch.randint(num_playlists, num_nodes, (1,))
        negs.append(rand.item())
    edge_index_negs = torch.row_stack([batch.edge_index[0,:], torch.LongTensor(negs)])
    return Data(edge_index=edge_index_negs)

## Training the model
Now, all that is left to do is train the model. Let's define a train function that goes through batches of edges and train the model while doing periodic evaluation using message passing edges.

Let's talk about the model we are using for training a recommendation system.

LightGCN is a state-of-the-art Graph Neural Network for collaborative filtering.
LightGCN learns user-item embeddings by propagation them on the graph and applies a weighted sum to obtain a final embedding of a node (user or item).

LightGCN has outperformed other GNN recommendation models, namely NGCF. In fact, LightGCN removed all learnable parameters from the traditional NGCF and replaced them with learnable parameters within the shallow embeddings of input nodes.

In [ ]:
def train(model, train_loader, train_mp, val_loader, val_mp, num_nodes, num_playlists, optimizer, device, k, n_epochs):
  running_loss = 0.
  total_edges = 0
  train_mp = train_mp.to(device)
  val_mp = val_mp.to(device)

  for epoch in range(1, n_epochs + 1):
    
    model.train() # Training
    with tqdm(train_loader, unit = 'batch', desc = 'Epoch [{:>2}/{:>2}] - {:>30}'.format(epoch, n_epochs, 'Training')) as tbatch:
      for i, batch in enumerate(tbatch):
        opt.zero_grad()
        negatives = sample_negative_edges(batch, num_playlists, num_nodes)
        batch, negatives = batch.to(device), negatives.to(device)
        # Inference
        pos_rankings = model(train_mp.edge_index, batch.edge_index) # Positive samples
        neg_rankings = model(train_mp.edge_index, negatives.edge_index) # Negative samples
        # Loss calculation
        loss = model.recommendation_loss(pos_rankings, neg_rankings)
        # Loss Backpropagation
        loss.backward()
        opt.step()
        # Gather and report
        running_loss += loss.item()*pos_rankings.size(0)*batch.edge_index.shape[1]
        total_edges += batch.edge_index.shape[1]
        if i == len(tbatch) - 1:
          avg_loss = running_loss/total_edges
          tbatch.set_postfix_str('Training loss = {:.5f}'.format(avg_loss))
    
    model.eval() # Validation
    recalls_dict = {}
    with torch.no_grad():
      with tqdm(val_loader, unit = 'batch', desc = 'Epoch [{:>2}/{:>2}] - {:>30}'.format(epoch, n_epochs, 'Validation')) as vbatch:
        for j, batch in enumerate(vbatch):
          batch = batch.to(device)
          # Making prediction (Top k songs predicted by model for each playlist)
          top_k = model.recommend(val_mp.edge_index, torch.unique(batch.edge_index[0, :]), torch.unique(batch.edge_index[1, :]), k)
          # Calculating recall@k
          unique_playlists = torch.unique(batch.edge_index[0, :])
          for i, pidx in enumerate(unique_playlists):
            ground_truth = val_mp.edge_index[1, val_mp.edge_index[0, :] == pidx].cpu()
            preds = top_k[i].cpu()
            recall = len(np.intersect1d(ground_truth, preds))/len(ground_truth)
            recalls_dict[pidx] = recall
          if j == len(vbatch) - 1:
            vbatch.set_postfix_str('Validation recall = {:.5f}'.format(np.mean(list(recalls_dict.values()))))

  return avg_loss

Let's gather everything, push data to GPU and start training.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = LightGCN(num_nodes = data.num_nodes, embedding_dim = 64, num_layers = 3)
model.to(device)

opt = torch.optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
train(model, train_loader, train_mp, val_loader, val_mp, data.num_nodes, len(playlist_dict), opt, device, 300, 10)

Epoch [ 1/10] -                       Training:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [ 1/10] -                     Validation:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [ 2/10] -                       Training:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [ 2/10] -                     Validation:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [ 3/10] -                       Training:   0%|          | 0/39 [00:00<?, ?batch/s]

...

Epoch [ 9/10] -                       Training:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [ 9/10] -                     Validation:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [10/10] -                       Training:   0%|          | 0/39 [00:00<?, ?batch/s]

Epoch [10/10] -                     Validation:   0%|          | 0/39 [00:00<?, ?batch/s]

0.6952129622238029